In [1]:
using Base.Threads
using Distributions
using LaTeXStrings
using Printf
using LsqFit
using BenchmarkTools
using LinearAlgebra

In [2]:
nthreads()

2

In [3]:
EE = rand(20000)
E = reshape(EE,2,100,100)

function prueba_vel_ji!(EE)
    @inbounds E = reshape(EE,2,100,100)
    @inbounds for i in 1:100
        @inbounds    for j in 1:100
            E[1,j,i] = 5.0 * E[2,j,i]
        end
    end
end

function prueba_vel_ij!(EE)
    @inbounds E = reshape(EE,2,100,100)
    @inbounds @threads for i in 1:100
        @inbounds    for j in 1:100
            E[1,i,j] = 5.0 * E[2,i,j]
        end
    end
end

function prueba_vel_no_for!(EE)
    @inbounds E = reshape(EE,2,100,100)   
    @inbounds E[1,:,:] .= 5.0 * E[2,:,:]
    return 0
end

@show @btime prueba_vel_ij!($EE)
@show @btime prueba_vel_ji!($EE)
@show @btime prueba_vel_no_for!($EE)

#sum(reshape(EE,2,100,100)[1,:,:] - 5*reshape(EE,2,100,100)[2,:,:])


  12.786 μs (14 allocations: 1.09 KiB)
#= /Users/reula/Julia/PIC/PIC-1D/Tests/speed_test.ipynb:28 =# @btime(prueba_vel_ij!($(Expr(:$, :EE)))) = nothing
  

3.916 μs (2 allocations: 96 bytes)
#= /Users/reula/Julia/PIC/PIC-1D/Tests/speed_test.ipynb:29 =# @btime(prueba_vel_ji!($(Expr(:$, :EE)))) = nothing


  23.232 μs (6 allocations: 156.44 KiB)
#= /Users/reula/Julia/PIC/PIC-1D/Tests/speed_test.ipynb:30 =# @btime(prueba_vel_no_for!($(Expr(:$, :EE)))) = 0


0

12.786 μs (14 allocations: 1.09 KiB) @btime(prueba_vel_ij!($(Expr(:$, :EE))))  con threads(2)

3.916 μs (2 allocations: 96 bytes) @btime(prueba_vel_ji!($(Expr(:$, :EE)))) = nothing

23.232 μs (6 allocations: 156.44 KiB) @btime(prueba_vel_no_for!($(Expr(:$, :EE)))) = 0


In [11]:
sum(reshape(EE,2,100,100)[1,:,:] - 5*reshape(EE,2,100,100)[2,:,:])

0.0

In [4]:
function prueba_vel_ji!(EE)
    @inbounds E = reshape(EE,100,100,2)
    @inbounds for i in 1:100
        @inbounds    for j in 1:100
            E[j,i,1] = 5.0 * E[j,i,2]
        end
    end
end

function prueba_vel_ij!(EE)
    @inbounds E = reshape(EE,100,100,2)
    @inbounds @threads for i in 1:100
        @inbounds    for j in 1:100
            E[i,j,1] = 5.0 * E[i,j,1]
        end
    end
end

function prueba_vel_no_for!(EE)
    @inbounds E = reshape(EE,2,100,100)   
    @inbounds E[1,:,:] .= 5.0 * E[2,:,:]
    #return 0
end

@btime prueba_vel_ij!($EE)
@btime prueba_vel_ji!($EE)
@btime prueba_vel_no_for!($EE)

  10.943 μs (14 allocations: 1.09 KiB)


  1.831 μs (2 allocations: 96 bytes)


  23.328 μs (6 allocations: 156.44 KiB)


100×100 view(::Array{Float64, 3}, 1, :, :) with eltype Float64:
  9.20785   19.4124   16.5751    12.5862   …  4.55853   2.98121   4.05397
  2.05765    7.92857   0.176943  19.4907      1.66135   2.94697   3.86022
  8.72828    3.54347   6.75452   14.293       2.53011   1.91936   1.18872
 11.8086     6.85313  22.4195    19.3627      0.252382  4.51313   4.69006
 15.2136    15.2441   24.5434    10.5048      3.11732   1.7134    0.192263
 10.469     14.468     3.08793    1.32426  …  3.68172   1.16937   4.77546
 16.8052    16.4497   16.5469    14.9361      3.23405   1.16687   2.71767
  0.378056  20.8655    4.37789   24.2992      0.865079  0.459007  0.856628
  8.3643    22.6094    8.85337   22.9369      2.41046   0.621267  2.53018
 16.4141     9.43977   1.97418    2.50639     4.76842   2.86662   1.9559
  ⋮                                        ⋱                      
  6.01244    7.44665  11.5307    20.5556      4.02438   2.43362   3.58655
 14.6064    21.1412    1.04108   19.314       0.167876

10.943 μs (14 allocations: 1.09 KiB) con threads ij

1.831 μs (2 allocations: 96 bytes) ji

23.328 μs (6 allocations: 156.44 KiB) ::,::